In [ ]:
## This file implements neural networks with/without dropout and regularizer for pyopresabsSTCC_qual with four replicates.
## We compute the mean and standarad deviation of training and test accuracies.
## We also compute the mean and standard deviation of AUC ROC values for each model.

In [1]:
from numpy.random import seed
import numpy as np
seed(100)
import tensorflow
tensorflow.random.set_seed(123)

In [2]:
import pandas as pd

df = pd.read_csv('/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/pyopresabsSTCC_qual.csv')
df.shape

(253, 614)

In [3]:
df.rename(columns={'Unnamed: 0':'id'}, inplace=True)

In [4]:
df['pheno']

0      0
1      0
2      0
3      0
4      0
      ..
248    0
249    0
250    0
251    0
252    0
Name: pheno, Length: 253, dtype: int64

In [5]:
df.head()

,id,TTTTCCCCCAT,TTCAAATTACCCTCTCTTGTAAAATCAAATTCACATGATGTCCATGGTTC,TGGGTCTGAC,TCCTGATGGACCAAAACCTAATTTAATCCAATCTATATAATCAAACGATACTTTCAAATTACCCTCTCTTGTAAAATCAAATTCACATGATGTCCATGGT,TATATAGACTG,TAGTCGCACT,TAAGAATAATATATTAAATATTTATTAACAAATTATAGATAAAATATGAATAATTAATTAATGGTATTTACATATTCATAACC,GGGCTGAGG,GAGCAACCTT,...,group_8646,group_8815,group_8892,group_9007,group_9104,group_9110,group_9207,ST,CC,pheno
0,107,0,1,1,1,1,0,1,0,0,...,0,0,0,0,0,0,0,5,5,0
1,109,0,1,0,1,1,0,1,0,0,...,0,0,0,0,0,0,0,8,8,0
2,115,0,0,0,0,1,1,1,1,1,...,0,0,0,0,0,0,0,5,5,0
3,120335,0,1,0,1,1,0,1,0,0,...,0,0,0,0,0,0,0,5,5,0
4,120337,0,1,0,1,1,0,1,0,0,...,0,0,0,0,0,0,0,5,5,0


In [6]:
df['pheno'].value_counts()

0    217
1     32
2      4
Name: pheno, dtype: int64

In [7]:
df_clean = df.drop(columns=['id'])

In [8]:
df_clean.shape

(253, 613)

In [9]:
df_clean.head()

,TTTTCCCCCAT,TTCAAATTACCCTCTCTTGTAAAATCAAATTCACATGATGTCCATGGTTC,TGGGTCTGAC,TCCTGATGGACCAAAACCTAATTTAATCCAATCTATATAATCAAACGATACTTTCAAATTACCCTCTCTTGTAAAATCAAATTCACATGATGTCCATGGT,TATATAGACTG,TAGTCGCACT,TAAGAATAATATATTAAATATTTATTAACAAATTATAGATAAAATATGAATAATTAATTAATGGTATTTACATATTCATAACC,GGGCTGAGG,GAGCAACCTT,GAACCATGGACATCATGTGAATTTGATTTTACAAGAGAGGGT,...,group_8646,group_8815,group_8892,group_9007,group_9104,group_9110,group_9207,ST,CC,pheno
0,0,1,1,1,1,0,1,0,0,1,...,0,0,0,0,0,0,0,5,5,0
1,0,1,0,1,1,0,1,0,0,1,...,0,0,0,0,0,0,0,8,8,0
2,0,0,0,0,1,1,1,1,1,0,...,0,0,0,0,0,0,0,5,5,0
3,0,1,0,1,1,0,1,0,0,1,...,0,0,0,0,0,0,0,5,5,0
4,0,1,0,1,1,0,1,0,0,1,...,0,0,0,0,0,0,0,5,5,0


In [10]:
X = df.loc[:, df.columns != 'pheno']
y = df['pheno']
print(X.shape, y.shape)

(253, 613) (253,)


In [11]:
# over-sampling
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
overS = RandomOverSampler(random_state=100)
X_over, y_over = overS.fit_resample(X, y)
print(sorted(Counter(y_over).items()))

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.ensemble.bagging module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)
/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.ensemble.base module is  deprecated in versi

[(0, 217), (1, 217), (2, 217)]


/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.utils.testing module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.utils. Anything that cannot be imported from sklearn.utils is now part of the private API.
  warnings.warn(message, FutureWarning)
/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is depreca

In [12]:
############# Fully-Connected Neural Network ################

In [13]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.regularizers import l1

In [14]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_over)

In [15]:
dat = pd.DataFrame(X_test_over[:,0])
dat['test'] = y_test_over

In [16]:
dat

,0,test
0,CFBRSa07,0
1,CFBRSa66A,0
2,NRS112,1
3,NRS211,0
4,CFBRSa22,0
...,...,...
191,NRS148,2
192,NRS255,2
193,NRS205,2
194,NRS255,2


In [17]:
X_train_over = X_train_over[:,1:]
X_test_over = X_test_over[:,1:]

In [18]:
#### neural network on over-sampling data
model1_over = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [19]:
model1_over.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [20]:
model1_over.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 455 samples, validate on 196 samples
Epoch 1/100
455/455 [==============================] - 0s 300us/step - loss: 3.0195 - accuracy: 0.5099 - val_loss: 1.0229 - val_accuracy: 0.5357
Epoch 2/100
455/455 [==============================] - 0s 101us/step - loss: 0.9292 - accuracy: 0.5451 - val_loss: 0.7798 - val_accuracy: 0.6378
Epoch 3/100
455/455 [==============================] - 0s 113us/step - loss: 0.5511 - accuracy: 0.7626 - val_loss: 0.4758 - val_accuracy: 0.7347
Epoch 4/100
455/455 [==============================] - 0s 114us/step - loss: 0.3975 - accuracy: 0.8527 - val_loss: 0.3731 - val_accuracy: 0.8724
Epoch 5/100
455/455 [==============================] - 0s 117us/step - loss: 0.3076 - accuracy: 0.9033 - val_loss: 0.3138 - val_accuracy: 0.9235
Epoch 6/100
455/455 [==============================] - 0s 115us/step - loss: 0.2573 - accuracy: 0.9297 - val_loss: 0.2655 - val_accuracy: 0.9337
Epoch 7/100
455/455 [==============================] - 0s 108us/step - loss: 0.2241 

Epoch 57/100
455/455 [==============================] - 0s 89us/step - loss: 0.0128 - accuracy: 1.0000 - val_loss: 0.0763 - val_accuracy: 0.9694
Epoch 58/100
455/455 [==============================] - 0s 82us/step - loss: 0.0125 - accuracy: 1.0000 - val_loss: 0.0742 - val_accuracy: 0.9694
Epoch 59/100
455/455 [==============================] - 0s 83us/step - loss: 0.0117 - accuracy: 1.0000 - val_loss: 0.0748 - val_accuracy: 0.9694
Epoch 60/100
455/455 [==============================] - 0s 80us/step - loss: 0.0112 - accuracy: 1.0000 - val_loss: 0.0760 - val_accuracy: 0.9694
Epoch 61/100
455/455 [==============================] - 0s 84us/step - loss: 0.0110 - accuracy: 1.0000 - val_loss: 0.0749 - val_accuracy: 0.9694
Epoch 62/100
455/455 [==============================] - 0s 97us/step - loss: 0.0106 - accuracy: 1.0000 - val_loss: 0.0750 - val_accuracy: 0.9694
Epoch 63/100
455/455 [==============================] - 0s 83us/step - loss: 0.0102 - accuracy: 1.0000 - val_loss: 0.0765 - val_ac

In [21]:
acc_test_over = model1_over.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over*100))

196/196 [==============================] - 0s 86us/step
over-sampling test accuracy: 96.94%


In [21]:
pred = model1_over.predict_classes(X_test_over)
pred

array([0, 0, 1, 1, 0, 1, 1, 0, 2, 1, 0, 0, 1, 2, 0, 0, 1, 0, 0, 1, 1, 1,
       0, 1, 0, 2, 0, 0, 0, 2, 2, 0, 2, 0, 0, 0, 2, 2, 1, 2, 1, 0, 0, 0,
       0, 1, 0, 1, 1, 1, 2, 2, 2, 1, 1, 1, 0, 1, 1, 1, 2, 0, 0, 2, 0, 1,
       0, 1, 2, 0, 0, 1, 0, 2, 2, 1, 1, 2, 0, 1, 0, 0, 0, 1, 0, 1, 2, 1,
       0, 0, 0, 1, 1, 2, 2, 2, 2, 2, 1, 1, 2, 0, 0, 0, 1, 2, 1, 0, 2, 1,
       1, 1, 0, 2, 1, 2, 2, 1, 1, 2, 1, 2, 2, 2, 2, 1, 1, 0, 2, 1, 0, 0,
       0, 2, 2, 1, 2, 0, 0, 0, 2, 1, 0, 2, 1, 0, 1, 2, 2, 1, 1, 2, 2, 2,
       2, 2, 2, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 2, 0, 2, 1, 2, 2, 2,
       1, 1, 2, 1, 0, 2, 1, 2, 2, 2, 2, 1, 2, 1, 0, 2, 2, 2, 2, 2])

In [22]:
dat['pred'] = pred
dat

,0,test,pred
0,CFBRSa07,0,0
1,CFBRSa66A,0,0
2,NRS112,1,1
3,NRS211,0,1
4,CFBRSa22,0,0
...,...,...,...
191,NRS148,2,2
192,NRS255,2,2
193,NRS205,2,2
194,NRS255,2,2


In [23]:
proba1 = model1_over.predict_proba(X_test_over)
dat_proba1 = pd.DataFrame(proba1)

In [24]:
dat_proba1

,0,1,2
0,0.986537,1.338227e-02,8.061646e-05
1,1.000000,1.343655e-07,1.434302e-10
2,0.001014,9.989378e-01,4.854937e-05
3,0.359725,6.401317e-01,1.429325e-04
4,0.995003,4.996928e-03,5.317322e-08
...,...,...,...
191,0.000007,9.932064e-05,9.998934e-01
192,0.000257,2.048159e-03,9.976944e-01
193,0.000011,4.546218e-05,9.999435e-01
194,0.000257,2.048159e-03,9.976944e-01


In [25]:
dat_proba1.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba1.csv", index = False,
         header=None)

In [26]:
dat.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/1pyopST.csv", index = False,
         header=None)

In [25]:
hist1_over = model1_over.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 455 samples, validate on 196 samples
Epoch 1/100
455/455 [==============================] - 0s 160us/step - loss: 0.0034 - accuracy: 1.0000 - val_loss: 0.0754 - val_accuracy: 0.9745
Epoch 2/100
455/455 [==============================] - 0s 147us/step - loss: 0.0033 - accuracy: 1.0000 - val_loss: 0.0764 - val_accuracy: 0.9745
Epoch 3/100
455/455 [==============================] - 0s 156us/step - loss: 0.0032 - accuracy: 1.0000 - val_loss: 0.0784 - val_accuracy: 0.9745
Epoch 4/100
455/455 [==============================] - 0s 153us/step - loss: 0.0032 - accuracy: 1.0000 - val_loss: 0.0784 - val_accuracy: 0.9745
Epoch 5/100
455/455 [==============================] - 0s 148us/step - loss: 0.0031 - accuracy: 1.0000 - val_loss: 0.0750 - val_accuracy: 0.9745
Epoch 6/100
455/455 [==============================] - 0s 143us/step - loss: 0.0031 - accuracy: 1.0000 - val_loss: 0.0760 - val_accuracy: 0.9745
Epoch 7/100
455/455 [==============================] - 0s 155us/step - loss: 0.0030 

Epoch 57/100
455/455 [==============================] - 0s 122us/step - loss: 0.0012 - accuracy: 1.0000 - val_loss: 0.0852 - val_accuracy: 0.9694
Epoch 58/100
455/455 [==============================] - 0s 111us/step - loss: 0.0012 - accuracy: 1.0000 - val_loss: 0.0867 - val_accuracy: 0.9694
Epoch 59/100
455/455 [==============================] - 0s 99us/step - loss: 0.0012 - accuracy: 1.0000 - val_loss: 0.0864 - val_accuracy: 0.9694
Epoch 60/100
455/455 [==============================] - 0s 101us/step - loss: 0.0011 - accuracy: 1.0000 - val_loss: 0.0868 - val_accuracy: 0.9694
Epoch 61/100
455/455 [==============================] - 0s 105us/step - loss: 0.0011 - accuracy: 1.0000 - val_loss: 0.0878 - val_accuracy: 0.9694
Epoch 62/100
455/455 [==============================] - 0s 98us/step - loss: 0.0011 - accuracy: 1.0000 - val_loss: 0.0878 - val_accuracy: 0.9694
Epoch 63/100
455/455 [==============================] - 0s 93us/step - loss: 0.0011 - accuracy: 1.0000 - val_loss: 0.0884 - va

In [26]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over.history['accuracy'])*100))

over-sampling train accuracy: 100.00%


In [18]:
df_proba = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_2.xlsx",
                        sheet_name=0,
                        index_col=None)

In [19]:
df_proba

,phage,strain,phenotype,prediction,0,1,2
0,p002ykpresabs_qual,CFBRSa26,0,0,0.758914,0.241086,4.638713e-07
1,p002ykpresabs_qual,NRS109,2,2,0.005361,0.016236,9.784034e-01
2,p002ykpresabs_qual,NRS112,0,0,0.726623,0.273376,1.520979e-06
3,p002ykpresabs_qual,NRS216,1,1,0.138322,0.861665,1.334123e-05
4,p002ykpresabs_qual,NRS021,0,0,0.882176,0.117824,1.414530e-10
...,...,...,...,...,...,...,...
4279,pyopresabsSTCC_qual,NRS148,2,2,0.000007,0.000099,9.998934e-01
4280,pyopresabsSTCC_qual,NRS255,2,2,0.000257,0.002048,9.976944e-01
4281,pyopresabsSTCC_qual,NRS205,2,2,0.000011,0.000045,9.999435e-01
4282,pyopresabsSTCC_qual,NRS255,2,2,0.000257,0.002048,9.976944e-01


In [20]:
y_prob = df_proba[df_proba['phage']=='pyopresabsSTCC_qual'].iloc[:,-3:]
y_prob = y_prob.to_numpy()
y_prob

array([[9.86537100e-01, 1.33822650e-02, 8.06164600e-05],
       [9.99999900e-01, 1.34365450e-07, 1.43430180e-10],
       [1.01362180e-03, 9.98937800e-01, 4.85493700e-05],
       [3.59725400e-01, 6.40131700e-01, 1.42932490e-04],
       [9.95003040e-01, 4.99692800e-03, 5.31732240e-08],
       [1.07127470e-02, 9.89278300e-01, 8.93336600e-06],
       [2.95426680e-02, 9.68566100e-01, 1.89115150e-03],
       [9.84435700e-01, 1.55543980e-02, 9.91228350e-06],
       [1.10352210e-05, 4.54621800e-05, 9.99943500e-01],
       [1.13115260e-03, 9.98444860e-01, 4.23919760e-04],
       [9.99902500e-01, 9.75163000e-05, 3.20035230e-08],
       [9.99900940e-01, 9.90262050e-05, 1.43656520e-08],
       [4.58405380e-04, 9.98622800e-01, 9.18754300e-04],
       [1.10352210e-05, 4.54621800e-05, 9.99943500e-01],
       [1.00000000e+00, 7.75336700e-13, 8.94346700e-12],
       [9.99954100e-01, 4.58226820e-05, 1.34741680e-07],
       [1.63219780e-03, 9.98350260e-01, 1.76124690e-05],
       [9.97122700e-01, 2.87675

In [21]:
## Retrieved from https://github.com/scikit-learn/scikit-learn/issues/3298
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelBinarizer

def rocauc_ovo(truth, pred, average="macro", multi_class="ovo"):

    lb = LabelBinarizer()
    lb.fit(truth)

    truth = lb.transform(truth)   
    
    return roc_auc_score(truth, pred, average=average, multi_class=multi_class)

In [22]:
ovo1 = rocauc_ovo(y_test_over, y_prob, average="macro", multi_class="ovo")
ovo1

0.9987473086709727

In [23]:
def rocauc_ovr(truth, pred, average="macro", multi_class="ovr"):

    lb = LabelBinarizer()
    lb.fit(truth)

    truth = lb.transform(truth)   

    return roc_auc_score(truth, pred, average=average, multi_class=multi_class)

In [24]:
ovr1 = rocauc_ovr(y_test_over, y_prob, average="macro", multi_class="ovr")
ovr1

0.9987473086709727

In [25]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=234,
                                                    stratify=y_over)

In [26]:
dat2 = pd.DataFrame(X_test_over[:,0])
dat2['test'] = y_test_over

In [27]:
dat2

,0,test
0,BCH-SA-04,0
1,NRS110,1
2,NRS109,2
3,NRS183,1
4,BCH-SA-05,0
...,...,...
191,NRS112,1
192,SR1065,0
193,NRS203,0
194,CFBREBSa129,0


In [28]:
X_train_over = X_train_over[:,1:]
X_test_over = X_test_over[:,1:]

In [31]:
model1_over2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [32]:
model1_over2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [33]:
model1_over2.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 455 samples, validate on 196 samples
Epoch 1/100
455/455 [==============================] - 0s 386us/step - loss: 9.8579 - accuracy: 0.5538 - val_loss: 3.5862 - val_accuracy: 0.6071
Epoch 2/100
455/455 [==============================] - 0s 196us/step - loss: 3.0661 - accuracy: 0.5846 - val_loss: 1.5635 - val_accuracy: 0.4694
Epoch 3/100
455/455 [==============================] - 0s 183us/step - loss: 1.0319 - accuracy: 0.6176 - val_loss: 0.5851 - val_accuracy: 0.6684
Epoch 4/100
455/455 [==============================] - 0s 161us/step - loss: 0.4823 - accuracy: 0.7758 - val_loss: 0.4528 - val_accuracy: 0.8163
Epoch 5/100
455/455 [==============================] - 0s 200us/step - loss: 0.3625 - accuracy: 0.8659 - val_loss: 0.3674 - val_accuracy: 0.8265
Epoch 6/100
455/455 [==============================] - 0s 226us/step - loss: 0.3052 - accuracy: 0.9011 - val_loss: 0.3299 - val_accuracy: 0.8980
Epoch 7/100
455/455 [==============================] - 0s 182us/step - loss: 0.2749 

Epoch 57/100
455/455 [==============================] - 0s 102us/step - loss: 0.0142 - accuracy: 1.0000 - val_loss: 0.0890 - val_accuracy: 0.9592
Epoch 58/100
455/455 [==============================] - 0s 150us/step - loss: 0.0139 - accuracy: 1.0000 - val_loss: 0.0912 - val_accuracy: 0.9643
Epoch 59/100
455/455 [==============================] - 0s 154us/step - loss: 0.0131 - accuracy: 1.0000 - val_loss: 0.0898 - val_accuracy: 0.9643
Epoch 60/100
455/455 [==============================] - 0s 109us/step - loss: 0.0126 - accuracy: 1.0000 - val_loss: 0.0898 - val_accuracy: 0.9643
Epoch 61/100
455/455 [==============================] - 0s 92us/step - loss: 0.0124 - accuracy: 1.0000 - val_loss: 0.0878 - val_accuracy: 0.9643
Epoch 62/100
455/455 [==============================] - 0s 91us/step - loss: 0.0118 - accuracy: 1.0000 - val_loss: 0.0884 - val_accuracy: 0.9643
Epoch 63/100
455/455 [==============================] - 0s 113us/step - loss: 0.0116 - accuracy: 1.0000 - val_loss: 0.0845 - v

In [34]:
acc_test_over2 = model1_over2.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over2*100))

196/196 [==============================] - 0s 103us/step
over-sampling test accuracy: 95.92%


In [34]:
pred2 = model1_over2.predict_classes(X_test_over)
pred2

array([0, 1, 2, 1, 0, 0, 2, 2, 2, 2, 1, 0, 1, 2, 0, 0, 2, 1, 2, 1, 2, 1,
       2, 1, 0, 1, 0, 1, 2, 2, 2, 1, 1, 0, 0, 2, 0, 2, 1, 1, 1, 1, 1, 2,
       1, 2, 1, 1, 2, 1, 2, 0, 1, 0, 0, 1, 2, 1, 0, 1, 2, 0, 1, 1, 1, 1,
       0, 2, 1, 2, 1, 2, 2, 0, 0, 0, 0, 1, 0, 1, 2, 1, 0, 2, 0, 1, 1, 1,
       2, 1, 1, 0, 0, 1, 0, 2, 0, 2, 0, 1, 2, 0, 2, 1, 0, 0, 1, 0, 2, 2,
       0, 2, 0, 2, 1, 2, 1, 2, 1, 2, 1, 1, 2, 1, 0, 1, 2, 1, 1, 1, 2, 2,
       0, 2, 0, 2, 2, 0, 1, 2, 0, 0, 1, 2, 2, 1, 1, 0, 0, 2, 2, 2, 1, 1,
       0, 2, 1, 2, 1, 0, 2, 0, 2, 1, 1, 1, 2, 1, 2, 0, 1, 0, 1, 2, 0, 2,
       1, 0, 0, 0, 0, 1, 2, 2, 1, 0, 2, 1, 2, 0, 2, 1, 0, 0, 0, 0])

In [35]:
dat2['pred'] = pred2
dat2

,0,test,pred
0,BCH-SA-04,0,0
1,NRS110,1,1
2,NRS109,2,2
3,NRS183,1,1
4,BCH-SA-05,0,0
...,...,...,...
191,NRS112,1,1
192,SR1065,0,0
193,NRS203,0,0
194,CFBREBSa129,0,0


In [36]:
proba2 = model1_over2.predict_proba(X_test_over)
dat_proba2 = pd.DataFrame(proba2)

In [37]:
dat_proba2

,0,1,2
0,0.999988,1.336353e-06,1.056695e-05
1,0.000292,9.997064e-01,1.785696e-06
2,0.000946,1.060771e-03,9.979930e-01
3,0.009562,9.864492e-01,3.988570e-03
4,0.999714,2.849912e-04,6.624472e-07
...,...,...,...
191,0.001860,9.979747e-01,1.653396e-04
192,0.982940,1.705227e-02,7.349168e-06
193,0.997093,1.962516e-03,9.441347e-04
194,1.000000,3.031141e-13,3.208205e-09


In [38]:
dat_proba2.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba2.csv", index = False,
         header=None)

In [39]:
dat2.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/2pyopST.csv", index = False,
         header=None)

In [38]:
hist1_over2 = model1_over2.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 455 samples, validate on 196 samples
Epoch 1/100
455/455 [==============================] - 0s 201us/step - loss: 0.0024 - accuracy: 1.0000 - val_loss: 0.0864 - val_accuracy: 0.9592
Epoch 2/100
455/455 [==============================] - 0s 125us/step - loss: 0.0024 - accuracy: 1.0000 - val_loss: 0.0834 - val_accuracy: 0.9592
Epoch 3/100
455/455 [==============================] - 0s 119us/step - loss: 0.0023 - accuracy: 1.0000 - val_loss: 0.0849 - val_accuracy: 0.9592
Epoch 4/100
455/455 [==============================] - 0s 104us/step - loss: 0.0023 - accuracy: 1.0000 - val_loss: 0.0862 - val_accuracy: 0.9592
Epoch 5/100
455/455 [==============================] - 0s 114us/step - loss: 0.0022 - accuracy: 1.0000 - val_loss: 0.0842 - val_accuracy: 0.9592
Epoch 6/100
455/455 [==============================] - 0s 140us/step - loss: 0.0022 - accuracy: 1.0000 - val_loss: 0.0844 - val_accuracy: 0.9592
Epoch 7/100
455/455 [==============================] - 0s 95us/step - loss: 0.0021 -

In [39]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over2.history['accuracy'])*100))

over-sampling train accuracy: 100.00%


In [29]:
df_proba2 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_2.xlsx",
                        sheet_name=1,
                        index_col=None)

In [30]:
df_proba2

,phage,strain,phenotype,prediction,0,1,2
0,p002ykpresabs_qual,NRS148,2,2,0.000056,1.748042e-03,9.981960e-01
1,p002ykpresabs_qual,BCH-SA-03,1,0,0.712007,2.879924e-01,9.646217e-07
2,p002ykpresabs_qual,NRS218,1,1,0.006222,9.937732e-01,4.482882e-06
3,p002ykpresabs_qual,NRS036,0,0,0.882617,1.173831e-01,2.310933e-10
4,p002ykpresabs_qual,NRS386,1,0,0.571179,4.288184e-01,2.444667e-06
...,...,...,...,...,...,...,...
4279,pyopresabsSTCC_qual,NRS112,1,1,0.001860,9.979747e-01,1.653396e-04
4280,pyopresabsSTCC_qual,SR1065,0,0,0.982940,1.705227e-02,7.349168e-06
4281,pyopresabsSTCC_qual,NRS203,0,0,0.997093,1.962516e-03,9.441347e-04
4282,pyopresabsSTCC_qual,CFBREBSa129,0,0,1.000000,3.031141e-13,3.208205e-09


In [31]:
y_prob2 = df_proba2[df_proba2['phage']=='pyopresabsSTCC_qual'].iloc[:,-3:]
y_prob2 = y_prob2.to_numpy()
y_prob2

array([[9.99988100e-01, 1.33635260e-06, 1.05669490e-05],
       [2.91759530e-04, 9.99706450e-01, 1.78569590e-06],
       [9.46207150e-04, 1.06077140e-03, 9.97993000e-01],
       [9.56231000e-03, 9.86449200e-01, 3.98856960e-03],
       [9.99714300e-01, 2.84991230e-04, 6.62447200e-07],
       [9.99937800e-01, 6.13005000e-05, 9.53952400e-07],
       [9.70995050e-04, 8.63280000e-04, 9.98165800e-01],
       [7.19664100e-05, 3.08167000e-05, 9.99897240e-01],
       [9.46207150e-04, 1.06077140e-03, 9.97993000e-01],
       [8.08099650e-05, 9.67242900e-06, 9.99909500e-01],
       [2.78682070e-04, 9.95044700e-01, 4.67671000e-03],
       [9.67313000e-01, 3.26792560e-02, 7.68869000e-06],
       [8.36012400e-03, 9.91557500e-01, 8.23392000e-05],
       [9.46207150e-04, 1.06077140e-03, 9.97993000e-01],
       [6.56932500e-01, 3.42625440e-01, 4.42099300e-04],
       [9.77757160e-01, 2.21037600e-02, 1.39100290e-04],
       [7.19664100e-05, 3.08167000e-05, 9.99897240e-01],
       [2.78682070e-04, 9.95044

In [32]:
ovo2 = rocauc_ovo(y_test_over, y_prob2, average="macro", multi_class="ovo")
ovo2

0.9998834498834498

In [33]:
ovr2 = rocauc_ovr(y_test_over, y_prob2, average="macro", multi_class="ovr")
ovr2

0.9998834498834498

In [34]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=345,
                                                    stratify=y_over)

In [35]:
dat3 = pd.DataFrame(X_test_over[:,0])
dat3['test'] = y_test_over

In [36]:
dat3

,0,test
0,NRS168,1
1,NRS383,1
2,NRS148,2
3,NRS109,2
4,NRS213,0
...,...,...
191,NRS255,2
192,NRS255,2
193,NRS266,1
194,NRS001,1


In [37]:
X_train_over = X_train_over[:,1:]
X_test_over = X_test_over[:,1:]

In [44]:
model1_over3 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [45]:
model1_over3.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [46]:
model1_over3.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 455 samples, validate on 196 samples
Epoch 1/100
455/455 [==============================] - 0s 358us/step - loss: 2.4737 - accuracy: 0.5165 - val_loss: 1.1082 - val_accuracy: 0.6327
Epoch 2/100
455/455 [==============================] - 0s 201us/step - loss: 0.7345 - accuracy: 0.6747 - val_loss: 0.6904 - val_accuracy: 0.7143
Epoch 3/100
455/455 [==============================] - 0s 203us/step - loss: 0.4872 - accuracy: 0.7912 - val_loss: 0.4596 - val_accuracy: 0.7908
Epoch 4/100
455/455 [==============================] - 0s 181us/step - loss: 0.3505 - accuracy: 0.8813 - val_loss: 0.3684 - val_accuracy: 0.8724
Epoch 5/100
455/455 [==============================] - 0s 205us/step - loss: 0.2803 - accuracy: 0.9231 - val_loss: 0.3025 - val_accuracy: 0.9133
Epoch 6/100
455/455 [==============================] - 0s 206us/step - loss: 0.2374 - accuracy: 0.9253 - val_loss: 0.2888 - val_accuracy: 0.8776
Epoch 7/100
455/455 [==============================] - 0s 190us/step - loss: 0.2115 

Epoch 57/100
455/455 [==============================] - 0s 108us/step - loss: 0.0105 - accuracy: 1.0000 - val_loss: 0.0712 - val_accuracy: 0.9796
Epoch 58/100
455/455 [==============================] - 0s 102us/step - loss: 0.0100 - accuracy: 1.0000 - val_loss: 0.0737 - val_accuracy: 0.9796
Epoch 59/100
455/455 [==============================] - 0s 157us/step - loss: 0.0096 - accuracy: 1.0000 - val_loss: 0.0678 - val_accuracy: 0.9796
Epoch 60/100
455/455 [==============================] - 0s 151us/step - loss: 0.0092 - accuracy: 1.0000 - val_loss: 0.0804 - val_accuracy: 0.9745
Epoch 61/100
455/455 [==============================] - 0s 98us/step - loss: 0.0090 - accuracy: 1.0000 - val_loss: 0.0721 - val_accuracy: 0.9796
Epoch 62/100
455/455 [==============================] - 0s 86us/step - loss: 0.0088 - accuracy: 1.0000 - val_loss: 0.0780 - val_accuracy: 0.9745
Epoch 63/100
455/455 [==============================] - 0s 85us/step - loss: 0.0084 - accuracy: 1.0000 - val_loss: 0.0770 - va

In [47]:
acc_test_over3 = model1_over3.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over3*100))

196/196 [==============================] - 0s 77us/step
over-sampling test accuracy: 97.96%


In [47]:
pred3 = model1_over3.predict_classes(X_test_over)
pred3

array([1, 1, 2, 2, 0, 0, 0, 2, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 2, 2, 1, 1,
       0, 2, 0, 1, 2, 0, 1, 0, 0, 2, 1, 2, 1, 1, 0, 2, 1, 1, 1, 2, 1, 2,
       0, 1, 2, 1, 1, 2, 0, 2, 1, 1, 0, 2, 0, 1, 2, 1, 2, 1, 0, 0, 2, 0,
       1, 0, 2, 1, 0, 0, 1, 0, 0, 2, 0, 1, 0, 0, 2, 0, 0, 2, 0, 2, 1, 1,
       2, 1, 2, 1, 0, 2, 0, 1, 2, 2, 1, 2, 2, 1, 2, 2, 1, 0, 2, 0, 2, 0,
       1, 2, 1, 1, 0, 1, 1, 2, 2, 1, 1, 2, 0, 2, 1, 1, 0, 1, 0, 2, 2, 0,
       1, 2, 2, 1, 2, 2, 2, 0, 1, 0, 0, 2, 1, 1, 1, 2, 0, 2, 2, 0, 2, 1,
       0, 1, 2, 0, 1, 1, 0, 0, 2, 0, 0, 0, 0, 2, 2, 0, 1, 1, 2, 0, 1, 2,
       2, 0, 2, 1, 0, 2, 0, 2, 1, 1, 1, 1, 1, 0, 2, 2, 2, 1, 1, 1])

In [48]:
dat3['pred'] = pred3
dat3

,0,test,pred
0,NRS168,1,1
1,NRS383,1,1
2,NRS148,2,2
3,NRS109,2,2
4,NRS213,0,0
...,...,...,...
191,NRS255,2,2
192,NRS255,2,2
193,NRS266,1,1
194,NRS001,1,1


In [49]:
proba3 = model1_over3.predict_proba(X_test_over)
dat_proba3 = pd.DataFrame(proba3)

In [50]:
dat_proba3

,0,1,2
0,0.000219,0.999493,2.874555e-04
1,0.006785,0.993205,1.003763e-05
2,0.000005,0.000123,9.998719e-01
3,0.000269,0.000652,9.990788e-01
4,0.996568,0.003003,4.286918e-04
...,...,...,...
191,0.000633,0.000928,9.984396e-01
192,0.000633,0.000928,9.984396e-01
193,0.025932,0.974061,7.323514e-06
194,0.000597,0.999403,3.675362e-10


In [51]:
dat_proba3.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba3.csv", index = False,
         header=None)

In [52]:
dat3.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/3pyopST.csv", index = False,
         header=None)

In [51]:
hist1_over3 = model1_over3.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 455 samples, validate on 196 samples
Epoch 1/100
455/455 [==============================] - 0s 267us/step - loss: 0.0041 - accuracy: 1.0000 - val_loss: 0.0752 - val_accuracy: 0.9796
Epoch 2/100
455/455 [==============================] - 0s 184us/step - loss: 0.0041 - accuracy: 1.0000 - val_loss: 0.0775 - val_accuracy: 0.9745
Epoch 3/100
455/455 [==============================] - 0s 220us/step - loss: 0.0041 - accuracy: 1.0000 - val_loss: 0.0685 - val_accuracy: 0.9847
Epoch 4/100
455/455 [==============================] - 0s 167us/step - loss: 0.0038 - accuracy: 1.0000 - val_loss: 0.0802 - val_accuracy: 0.9745
Epoch 5/100
455/455 [==============================] - 0s 336us/step - loss: 0.0038 - accuracy: 1.0000 - val_loss: 0.0761 - val_accuracy: 0.9796
Epoch 6/100
455/455 [==============================] - 0s 146us/step - loss: 0.0037 - accuracy: 1.0000 - val_loss: 0.0713 - val_accuracy: 0.9796
Epoch 7/100
455/455 [==============================] - 0s 138us/step - loss: 0.0035 

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/keras/callbacks/callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.100164). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


455/455 [==============================] - 0s 147us/step - loss: 0.0025 - accuracy: 1.0000 - val_loss: 0.0770 - val_accuracy: 0.9796
Epoch 24/100
455/455 [==============================] - 0s 181us/step - loss: 0.0024 - accuracy: 1.0000 - val_loss: 0.0756 - val_accuracy: 0.9796
Epoch 25/100
455/455 [==============================] - 0s 191us/step - loss: 0.0024 - accuracy: 1.0000 - val_loss: 0.0786 - val_accuracy: 0.9796
Epoch 26/100
455/455 [==============================] - 0s 127us/step - loss: 0.0023 - accuracy: 1.0000 - val_loss: 0.0750 - val_accuracy: 0.9796
Epoch 27/100
455/455 [==============================] - 0s 123us/step - loss: 0.0023 - accuracy: 1.0000 - val_loss: 0.0764 - val_accuracy: 0.9796
Epoch 28/100
455/455 [==============================] - 0s 134us/step - loss: 0.0023 - accuracy: 1.0000 - val_loss: 0.0794 - val_accuracy: 0.9796
Epoch 29/100
455/455 [==============================] - 0s 113us/step - loss: 0.0022 - accuracy: 1.0000 - val_loss: 0.0771 - val_accuracy

In [52]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over3.history['accuracy'])*100))

over-sampling train accuracy: 100.00%


In [38]:
df_proba3 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_2.xlsx",
                        sheet_name=2,
                        index_col=None)

In [39]:
df_proba3

,phage,strain,phenotype,prediction,0,1,2
0,p002ykpresabs_qual,NRS109,2,2,0.004477,0.013518,9.820048e-01
1,p002ykpresabs_qual,NRS109,2,2,0.004477,0.013518,9.820048e-01
2,p002ykpresabs_qual,NRS222,0,0,0.851725,0.148269,5.980786e-06
3,p002ykpresabs_qual,NRS109,2,2,0.004477,0.013518,9.820048e-01
4,p002ykpresabs_qual,GA50245,0,0,0.812055,0.187945,1.161034e-07
...,...,...,...,...,...,...,...
4279,pyopresabsSTCC_qual,NRS255,2,2,0.000633,0.000928,9.984396e-01
4280,pyopresabsSTCC_qual,NRS255,2,2,0.000633,0.000928,9.984396e-01
4281,pyopresabsSTCC_qual,NRS266,1,1,0.025932,0.974061,7.323514e-06
4282,pyopresabsSTCC_qual,NRS001,1,1,0.000597,0.999403,3.675362e-10


In [40]:
y_prob3 = df_proba3[df_proba3['phage']=='pyopresabsSTCC_qual'].iloc[:,-3:]
y_prob3 = y_prob3.to_numpy()
y_prob3

array([[2.19203110e-04, 9.99493240e-01, 2.87455540e-04],
       [6.78508000e-03, 9.93204830e-01, 1.00376340e-05],
       [5.17537460e-06, 1.22981990e-04, 9.99871850e-01],
       [2.68918000e-04, 6.52354600e-04, 9.99078750e-01],
       [9.96567850e-01, 3.00334300e-03, 4.28691830e-04],
       [9.96088400e-01, 3.91160230e-03, 2.38839760e-08],
       [1.00000000e+00, 1.40458230e-09, 4.14733900e-10],
       [2.68918000e-04, 6.52354600e-04, 9.99078750e-01],
       [1.00000000e+00, 1.30167830e-08, 7.95353960e-13],
       [9.31623570e-04, 9.99068440e-01, 1.19429160e-09],
       [9.99931340e-01, 4.54846100e-05, 2.31257820e-05],
       [9.66629270e-01, 3.33111550e-02, 5.96215070e-05],
       [2.53201820e-02, 9.74666540e-01, 1.32349800e-05],
       [9.34198560e-01, 6.56104500e-02, 1.90942850e-04],
       [8.73373700e-03, 9.91241460e-01, 2.48719260e-05],
       [1.91836470e-04, 9.99438700e-01, 3.69505200e-04],
       [9.99762500e-01, 7.14951800e-05, 1.66035780e-04],
       [9.99992600e-01, 6.57746

In [41]:
ovo3 = rocauc_ovo(y_test_over, y_prob3, average="macro", multi_class="ovo")
ovo3

0.9977383939215999

In [42]:
ovr3 = rocauc_ovr(y_test_over, y_prob3, average="macro", multi_class="ovr")
ovr3

0.9977383939215999

In [43]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=456,
                                                    stratify=y_over)

In [44]:
dat4 = pd.DataFrame(X_test_over[:,0])
dat4['test'] = y_test_over

In [45]:
dat4

,0,test
0,NRS178,1
1,NRS109,2
2,NRS073,1
3,CFBREBSa119,0
4,NRS109,2
...,...,...
191,NRS236,1
192,NRS029,0
193,NRS148,2
194,CFBRSa28,0


In [46]:
X_train_over = X_train_over[:,1:]
X_test_over = X_test_over[:,1:]

In [57]:
model1_over4 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [58]:
model1_over4.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [59]:
model1_over4.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 455 samples, validate on 196 samples
Epoch 1/100
455/455 [==============================] - 0s 426us/step - loss: 7.4481 - accuracy: 0.3582 - val_loss: 2.8016 - val_accuracy: 0.4184
Epoch 2/100
455/455 [==============================] - 0s 138us/step - loss: 1.1363 - accuracy: 0.6022 - val_loss: 0.9019 - val_accuracy: 0.6582
Epoch 3/100
455/455 [==============================] - 0s 147us/step - loss: 0.6060 - accuracy: 0.7670 - val_loss: 0.4790 - val_accuracy: 0.8316
Epoch 4/100
455/455 [==============================] - 0s 148us/step - loss: 0.3737 - accuracy: 0.8703 - val_loss: 0.4075 - val_accuracy: 0.8469
Epoch 5/100
455/455 [==============================] - 0s 151us/step - loss: 0.2942 - accuracy: 0.8945 - val_loss: 0.3572 - val_accuracy: 0.8776
Epoch 6/100
455/455 [==============================] - 0s 142us/step - loss: 0.2510 - accuracy: 0.9297 - val_loss: 0.3048 - val_accuracy: 0.8980
Epoch 7/100
455/455 [==============================] - 0s 176us/step - loss: 0.2195 

Epoch 57/100
455/455 [==============================] - 0s 128us/step - loss: 0.0100 - accuracy: 1.0000 - val_loss: 0.0958 - val_accuracy: 0.9592
Epoch 58/100
455/455 [==============================] - 0s 149us/step - loss: 0.0096 - accuracy: 1.0000 - val_loss: 0.0948 - val_accuracy: 0.9592
Epoch 59/100
455/455 [==============================] - 0s 223us/step - loss: 0.0092 - accuracy: 1.0000 - val_loss: 0.0945 - val_accuracy: 0.9592
Epoch 60/100
455/455 [==============================] - 0s 130us/step - loss: 0.0089 - accuracy: 1.0000 - val_loss: 0.0994 - val_accuracy: 0.9592
Epoch 61/100
455/455 [==============================] - 0s 102us/step - loss: 0.0087 - accuracy: 1.0000 - val_loss: 0.0939 - val_accuracy: 0.9592
Epoch 62/100
455/455 [==============================] - 0s 87us/step - loss: 0.0082 - accuracy: 1.0000 - val_loss: 0.0950 - val_accuracy: 0.9592
Epoch 63/100
455/455 [==============================] - 0s 123us/step - loss: 0.0080 - accuracy: 1.0000 - val_loss: 0.0948 - 

In [60]:
acc_test_over4 = model1_over4.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over4*100))

196/196 [==============================] - 0s 78us/step
over-sampling test accuracy: 95.41%


In [60]:
pred4 = model1_over4.predict_classes(X_test_over)
pred4

array([1, 2, 1, 0, 2, 1, 1, 2, 2, 0, 2, 1, 1, 2, 0, 1, 0, 2, 2, 1, 2, 1,
       1, 1, 1, 0, 2, 2, 1, 1, 2, 1, 1, 2, 2, 0, 0, 0, 0, 1, 1, 2, 1, 0,
       1, 1, 2, 2, 0, 0, 2, 1, 2, 2, 1, 0, 1, 2, 0, 1, 1, 1, 2, 1, 2, 0,
       0, 1, 1, 2, 0, 0, 1, 2, 0, 1, 2, 1, 1, 0, 0, 2, 1, 2, 0, 0, 1, 0,
       0, 2, 2, 2, 2, 2, 1, 1, 1, 1, 0, 2, 0, 0, 2, 2, 1, 1, 1, 2, 0, 1,
       2, 2, 2, 0, 2, 2, 2, 1, 1, 0, 0, 2, 0, 1, 0, 1, 2, 2, 2, 0, 1, 0,
       2, 1, 2, 0, 1, 1, 1, 2, 2, 1, 0, 2, 0, 0, 1, 0, 0, 0, 1, 2, 2, 2,
       1, 1, 1, 2, 2, 0, 1, 2, 1, 0, 1, 2, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0,
       0, 2, 1, 1, 1, 0, 2, 0, 0, 0, 0, 2, 0, 2, 2, 1, 1, 2, 0, 2])

In [61]:
dat4['pred'] = pred4
dat4

,0,test,pred
0,NRS178,1,1
1,NRS109,2,2
2,NRS073,1,1
3,CFBREBSa119,0,0
4,NRS109,2,2
...,...,...,...
191,NRS236,1,1
192,NRS029,0,1
193,NRS148,2,2
194,CFBRSa28,0,0


In [62]:
proba4 = model1_over4.predict_proba(X_test_over)
dat_proba4 = pd.DataFrame(proba4)

In [63]:
dat_proba4

,0,1,2
0,0.000003,0.999986,0.000011
1,0.000089,0.000591,0.999319
2,0.004790,0.994912,0.000299
3,0.997035,0.002957,0.000008
4,0.000089,0.000591,0.999319
...,...,...,...
191,0.000052,0.999768,0.000180
192,0.322350,0.677496,0.000153
193,0.000006,0.000026,0.999968
194,0.999288,0.000176,0.000536


In [64]:
dat_proba4.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba4.csv", index = False,
         header=None)

In [65]:
dat4.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/4pyopST.csv", index = False,
         header=None)

In [64]:
hist1_over4 = model1_over4.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 455 samples, validate on 196 samples
Epoch 1/100
455/455 [==============================] - 0s 144us/step - loss: 0.0041 - accuracy: 1.0000 - val_loss: 0.1242 - val_accuracy: 0.9541
Epoch 2/100
455/455 [==============================] - 0s 133us/step - loss: 0.0039 - accuracy: 1.0000 - val_loss: 0.1253 - val_accuracy: 0.9541
Epoch 3/100
455/455 [==============================] - 0s 136us/step - loss: 0.0039 - accuracy: 1.0000 - val_loss: 0.1233 - val_accuracy: 0.9541
Epoch 4/100
455/455 [==============================] - 0s 125us/step - loss: 0.0038 - accuracy: 1.0000 - val_loss: 0.1267 - val_accuracy: 0.9541
Epoch 5/100
455/455 [==============================] - 0s 123us/step - loss: 0.0038 - accuracy: 1.0000 - val_loss: 0.1267 - val_accuracy: 0.9541
Epoch 6/100
455/455 [==============================] - 0s 120us/step - loss: 0.0037 - accuracy: 1.0000 - val_loss: 0.1208 - val_accuracy: 0.9541
Epoch 7/100
455/455 [==============================] - 0s 163us/step - loss: 0.0035 

Epoch 57/100
455/455 [==============================] - 0s 126us/step - loss: 0.0014 - accuracy: 1.0000 - val_loss: 0.1303 - val_accuracy: 0.9541
Epoch 58/100
455/455 [==============================] - 0s 123us/step - loss: 0.0014 - accuracy: 1.0000 - val_loss: 0.1284 - val_accuracy: 0.9541
Epoch 59/100
455/455 [==============================] - 0s 109us/step - loss: 0.0014 - accuracy: 1.0000 - val_loss: 0.1274 - val_accuracy: 0.9541
Epoch 60/100
455/455 [==============================] - 0s 98us/step - loss: 0.0014 - accuracy: 1.0000 - val_loss: 0.1278 - val_accuracy: 0.9541
Epoch 61/100
455/455 [==============================] - 0s 108us/step - loss: 0.0013 - accuracy: 1.0000 - val_loss: 0.1273 - val_accuracy: 0.9541
Epoch 62/100
455/455 [==============================] - 0s 91us/step - loss: 0.0013 - accuracy: 1.0000 - val_loss: 0.1325 - val_accuracy: 0.9541
Epoch 63/100
455/455 [==============================] - 0s 88us/step - loss: 0.0013 - accuracy: 1.0000 - val_loss: 0.1287 - va

In [65]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over4.history['accuracy'])*100))

over-sampling train accuracy: 100.00%


In [47]:
df_proba4 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_2.xlsx",
                        sheet_name=3,
                        index_col=None)

In [48]:
df_proba4

,phage,strain,phenotype,prediction,0,1,2
0,p002ykpresabs_qual,NRS110,1,1,0.000003,0.999997,5.870196e-13
1,p002ykpresabs_qual,NRS216,1,1,0.039254,0.960745,9.078969e-07
2,p002ykpresabs_qual,NRS386,1,1,0.326752,0.673248,1.061032e-07
3,p002ykpresabs_qual,CFBRSa25,0,0,0.611084,0.388916,7.664974e-07
4,p002ykpresabs_qual,BCH-SA-03,1,0,0.611084,0.388916,7.664974e-07
...,...,...,...,...,...,...,...
4279,pyopresabsSTCC_qual,NRS236,1,1,0.000052,0.999768,1.803156e-04
4280,pyopresabsSTCC_qual,NRS029,0,1,0.322350,0.677496,1.533154e-04
4281,pyopresabsSTCC_qual,NRS148,2,2,0.000006,0.000026,9.999682e-01
4282,pyopresabsSTCC_qual,CFBRSa28,0,0,0.999288,0.000176,5.361527e-04


In [49]:
y_prob4 = df_proba4[df_proba4['phage']=='pyopresabsSTCC_qual'].iloc[:,-3:]
y_prob4 = y_prob4.to_numpy()
y_prob4

array([[2.84750880e-06, 9.99986400e-01, 1.07121390e-05],
       [8.90664450e-05, 5.91487700e-04, 9.99319430e-01],
       [4.78956940e-03, 9.94911850e-01, 2.98612900e-04],
       [9.97035400e-01, 2.95661090e-03, 7.90630000e-06],
       [8.90664450e-05, 5.91487700e-04, 9.99319430e-01],
       [1.34019960e-03, 9.97478900e-01, 1.18082520e-03],
       [2.38043020e-03, 9.97093100e-01, 5.26401850e-04],
       [8.90664450e-05, 5.91487700e-04, 9.99319430e-01],
       [6.62489000e-06, 6.62595130e-06, 9.99986770e-01],
       [1.00000000e+00, 1.39188890e-15, 7.66704000e-29],
       [8.90664450e-05, 5.91487700e-04, 9.99319430e-01],
       [1.34019960e-03, 9.97478900e-01, 1.18082520e-03],
       [6.09711160e-04, 9.99390000e-01, 1.86577180e-07],
       [6.62489000e-06, 6.62595130e-06, 9.99986770e-01],
       [9.99939200e-01, 6.05122730e-05, 2.21072850e-07],
       [1.65521990e-02, 9.83377000e-01, 7.07603000e-05],
       [9.99999760e-01, 1.98321860e-07, 1.11691930e-08],
       [5.76195800e-06, 2.60178

In [50]:
ovo4 = rocauc_ovo(y_test_over, y_prob4, average="macro", multi_class="ovo")
ovo4

1.0

In [51]:
ovr4 = rocauc_ovr(y_test_over, y_prob4, average="macro", multi_class="ovr")
ovr4

1.0

In [52]:
ovos = [ovo1, ovo2, ovo3, ovo4]
np.mean(ovos)

0.9990922881190056

In [53]:
np.std(ovos)

0.0009222144698700393

In [54]:
ovrs = [ovr1, ovr2, ovr3, ovr4]
np.mean(ovrs)

0.9990922881190056

In [55]:
np.std(ovrs)

0.0009222144698700393

In [66]:
accs = [acc_test_over, acc_test_over2, acc_test_over3, acc_test_over4]

In [67]:
mean = np.mean(accs)
print('over-sampling test accuracy mean: %.2f%%' % (mean*100))

over-sampling test accuracy mean: 96.56%


In [68]:
std = np.std(accs)
print('over-sampling test accuracy standard deviation:', std)

over-sampling test accuracy standard deviation: 0.00979737510868992


In [69]:
accs_train = [np.mean(hist1_over.history['accuracy']), np.mean(hist1_over2.history['accuracy']), np.mean(hist1_over3.history['accuracy']),
             np.mean(hist1_over4.history['accuracy'])]

In [70]:
mean_train = np.mean(accs_train)
print('over-sampling train accuracy mean: %.2f%%' % (mean_train*100))

over-sampling train accuracy mean: 100.00%


In [71]:
std_train = np.std(accs_train)
print('over-sampling train accuracy standard deviation:', std_train)

over-sampling train accuracy standard deviation: 0.0


In [56]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=567,
                                                    stratify=y_over)

In [57]:
dat5 = pd.DataFrame(X_test_over[:,0])
dat5['test'] = y_test_over

In [58]:
dat5

,0,test
0,NRS255,2
1,NRS255,2
2,NRS386,1
3,NRS205,2
4,NRS205,2
...,...,...
191,BCH-SA-12,0
192,NRS049,0
193,NRS022,0
194,NRS236,1


In [59]:
X_train_over = X_train_over[:,1:]
X_test_over = X_test_over[:,1:]

In [70]:
#### add regularizer and dropout
model1_over5 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],), activity_regularizer=l1(0.001)),
    Dense(3, activation='softmax'),
    Dropout(0.2, ),
])

In [71]:
model1_over5.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [72]:
model1_over5.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 455 samples, validate on 196 samples
Epoch 1/100
455/455 [==============================] - 0s 633us/step - loss: 9.5239 - accuracy: 0.4549 - val_loss: 6.0171 - val_accuracy: 0.6122
Epoch 2/100
455/455 [==============================] - 0s 180us/step - loss: 7.7914 - accuracy: 0.5846 - val_loss: 4.2569 - val_accuracy: 0.7551
Epoch 3/100
455/455 [==============================] - 0s 205us/step - loss: 6.7254 - accuracy: 0.6242 - val_loss: 3.2978 - val_accuracy: 0.7959
Epoch 4/100
455/455 [==============================] - 0s 221us/step - loss: 5.3537 - accuracy: 0.6352 - val_loss: 2.6666 - val_accuracy: 0.7857
Epoch 5/100
455/455 [==============================] - 0s 200us/step - loss: 4.5151 - accuracy: 0.6462 - val_loss: 2.0162 - val_accuracy: 0.8265
Epoch 6/100
455/455 [==============================] - 0s 138us/step - loss: 3.9507 - accuracy: 0.6593 - val_loss: 1.5635 - val_accuracy: 0.8112
Epoch 7/100
455/455 [==============================] - 0s 141us/step - loss: 3.8197 

In [79]:
acc_test_over5 = model1_over5.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over5*100))

196/196 [==============================] - 0s 115us/step
over-sampling test accuracy: 96.94%


In [73]:
pred5 = model1_over5.predict_classes(X_test_over)
pred5

array([2, 2, 1, 2, 2, 1, 1, 0, 2, 2, 1, 2, 2, 1, 0, 0, 1, 0, 1, 2, 0, 1,
       0, 2, 0, 0, 1, 0, 2, 2, 2, 0, 1, 1, 2, 1, 1, 2, 0, 2, 1, 1, 1, 0,
       2, 0, 1, 0, 0, 1, 1, 1, 2, 2, 1, 0, 0, 2, 2, 2, 1, 0, 1, 2, 2, 1,
       0, 1, 2, 0, 1, 1, 0, 0, 2, 0, 1, 1, 2, 2, 2, 0, 1, 2, 0, 2, 1, 0,
       2, 0, 2, 0, 1, 1, 0, 1, 0, 2, 0, 1, 1, 0, 2, 1, 1, 1, 0, 1, 2, 2,
       1, 2, 0, 0, 1, 0, 2, 1, 2, 1, 1, 1, 0, 1, 2, 1, 2, 1, 0, 1, 0, 2,
       0, 0, 1, 1, 2, 0, 1, 0, 2, 1, 1, 2, 2, 0, 2, 0, 0, 0, 0, 2, 1, 2,
       1, 1, 2, 2, 0, 0, 1, 0, 2, 1, 0, 1, 2, 2, 0, 2, 0, 2, 2, 2, 2, 2,
       1, 1, 0, 1, 0, 1, 2, 1, 1, 2, 1, 0, 1, 1, 2, 0, 1, 0, 1, 2])

In [74]:
dat5['pred'] = pred5
dat5

,0,test,pred
0,NRS255,2,2
1,NRS255,2,2
2,NRS386,1,1
3,NRS205,2,2
4,NRS205,2,2
...,...,...,...
191,BCH-SA-12,0,0
192,NRS049,0,1
193,NRS022,0,0
194,NRS236,1,1


In [75]:
proba5 = model1_over5.predict_proba(X_test_over)
dat_proba5 = pd.DataFrame(proba5)

In [76]:
dat_proba5

,0,1,2
0,5.623269e-08,1.073980e-07,9.999999e-01
1,5.623269e-08,1.073980e-07,9.999999e-01
2,6.623193e-04,9.991296e-01,2.080995e-04
3,3.763637e-08,2.140663e-08,1.000000e+00
4,3.763637e-08,2.140663e-08,1.000000e+00
...,...,...,...
191,1.000000e+00,1.152503e-09,1.898730e-09
192,8.401357e-11,1.000000e+00,3.209735e-13
193,1.000000e+00,4.755084e-10,1.974275e-10
194,1.357345e-08,1.000000e+00,1.293117e-10


In [77]:
dat_proba5.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba5.csv", index = False,
         header=None)

In [78]:
dat5.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/5pyopST.csv", index = False,
         header=None)

In [83]:
hist1_over5 = model1_over5.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 455 samples, validate on 196 samples
Epoch 1/100
455/455 [==============================] - 0s 165us/step - loss: 1.0469 - accuracy: 0.7714 - val_loss: 0.6993 - val_accuracy: 0.9643
Epoch 2/100
455/455 [==============================] - 0s 155us/step - loss: 0.9741 - accuracy: 0.7912 - val_loss: 0.7155 - val_accuracy: 0.9694
Epoch 3/100
455/455 [==============================] - 0s 147us/step - loss: 0.8303 - accuracy: 0.8286 - val_loss: 0.7040 - val_accuracy: 0.9694
Epoch 4/100
455/455 [==============================] - 0s 133us/step - loss: 0.8342 - accuracy: 0.8044 - val_loss: 0.6916 - val_accuracy: 0.9694
Epoch 5/100
455/455 [==============================] - 0s 130us/step - loss: 0.8871 - accuracy: 0.8066 - val_loss: 0.6690 - val_accuracy: 0.9694
Epoch 6/100
455/455 [==============================] - 0s 129us/step - loss: 0.9368 - accuracy: 0.8176 - val_loss: 0.6588 - val_accuracy: 0.9643
Epoch 7/100
455/455 [==============================] - 0s 135us/step - loss: 0.8087 

Epoch 57/100
455/455 [==============================] - 0s 132us/step - loss: 0.6246 - accuracy: 0.8330 - val_loss: 0.6515 - val_accuracy: 0.9643
Epoch 58/100
455/455 [==============================] - 0s 123us/step - loss: 0.6816 - accuracy: 0.7802 - val_loss: 0.6431 - val_accuracy: 0.9643
Epoch 59/100
455/455 [==============================] - 0s 116us/step - loss: 0.6772 - accuracy: 0.8044 - val_loss: 0.6174 - val_accuracy: 0.9694
Epoch 60/100
455/455 [==============================] - 0s 115us/step - loss: 0.6311 - accuracy: 0.8088 - val_loss: 0.6149 - val_accuracy: 0.9643
Epoch 61/100
455/455 [==============================] - 0s 118us/step - loss: 0.7442 - accuracy: 0.7956 - val_loss: 0.6138 - val_accuracy: 0.9694
Epoch 62/100
455/455 [==============================] - 0s 111us/step - loss: 0.6764 - accuracy: 0.8242 - val_loss: 0.5744 - val_accuracy: 0.9745
Epoch 63/100
455/455 [==============================] - 0s 111us/step - loss: 0.6771 - accuracy: 0.8088 - val_loss: 0.5708 -

In [84]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over5.history['accuracy'])*100))

over-sampling train accuracy: 80.17%


In [60]:
df_proba5 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_regularizor_dropout_2.xlsx",
                        sheet_name=0,
                        index_col=None)

In [61]:
df_proba5

,phage,strain,phenotype,prediction,0,1,2
0,p002ykpresabsSTCC_qual,NRS241,1,1,1.342914e-03,9.986569e-01,2.348628e-07
1,p002ykpresabsSTCC_qual,NRS148,2,2,5.170289e-08,1.017893e-07,9.999999e-01
2,p002ykpresabsSTCC_qual,NRS255,1,1,1.780311e-07,9.999999e-01,2.544841e-12
3,p002ykpresabsSTCC_qual,NRS214,0,0,1.000000e+00,2.203547e-10,5.688883e-15
4,p002ykpresabsSTCC_qual,NRS148,2,2,5.170289e-08,1.017893e-07,9.999999e-01
...,...,...,...,...,...,...,...
1977,pyopresabsSTCC_qual,BCH-SA-12,0,0,1.000000e+00,1.152503e-09,1.898730e-09
1978,pyopresabsSTCC_qual,NRS049,0,1,8.401357e-11,1.000000e+00,3.209735e-13
1979,pyopresabsSTCC_qual,NRS022,0,0,1.000000e+00,4.755084e-10,1.974275e-10
1980,pyopresabsSTCC_qual,NRS236,1,1,1.357345e-08,1.000000e+00,1.293117e-10


In [62]:
y_prob5 = df_proba5[df_proba5['phage']=='pyopresabsSTCC_qual'].iloc[:,-3:]
y_prob5 = y_prob5.to_numpy()
y_prob5

array([[5.62326860e-08, 1.07397980e-07, 9.99999900e-01],
       [5.62326860e-08, 1.07397980e-07, 9.99999900e-01],
       [6.62319300e-04, 9.99129600e-01, 2.08099470e-04],
       [3.76363720e-08, 2.14066350e-08, 1.00000000e+00],
       [3.76363720e-08, 2.14066350e-08, 1.00000000e+00],
       [6.82506900e-08, 9.99999900e-01, 9.66216300e-10],
       [5.57879700e-03, 9.94394900e-01, 2.63683940e-05],
       [9.97977200e-01, 8.46653600e-04, 1.17607430e-03],
       [4.84955800e-08, 9.32821100e-08, 9.99999900e-01],
       [4.07470430e-08, 2.32920120e-08, 9.99999900e-01],
       [2.39172070e-07, 9.99999760e-01, 2.77114180e-09],
       [3.76363720e-08, 2.14066350e-08, 1.00000000e+00],
       [4.07470430e-08, 2.32920120e-08, 9.99999900e-01],
       [2.35414600e-06, 9.99997140e-01, 4.81304700e-07],
       [9.99994900e-01, 3.51247330e-06, 1.69392060e-06],
       [9.97303370e-01, 2.66711270e-03, 2.95380890e-05],
       [4.91842700e-08, 1.00000000e+00, 3.30945900e-08],
       [1.00000000e+00, 1.74692

In [63]:
ovo5 = rocauc_ovo(y_test_over, y_prob5, average="macro", multi_class="ovo")
ovo5

0.9902812624186671

In [64]:
ovr5 = rocauc_ovr(y_test_over, y_prob5, average="macro", multi_class="ovr")
ovr5

0.9902812624186671

In [65]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=678,
                                                    stratify=y_over)

In [66]:
dat6 = pd.DataFrame(X_test_over[:,0])
dat6['test'] = y_test_over

In [67]:
dat6

,0,test
0,120335,0
1,NRS168,1
2,NRS202,0
3,NRS109,2
4,BCH-SA-01,0
...,...,...
191,NRS272,0
192,NRS112,1
193,NRS064,1
194,BCH-SA-04,0


In [68]:
X_train_over = X_train_over[:,1:]
X_test_over = X_test_over[:,1:]

In [83]:
model1_over6 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],), activity_regularizer=l1(0.001)),
    Dense(3, activation='softmax'),
    Dropout(0.2, ),
])

In [84]:
model1_over6.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [85]:
model1_over6.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 455 samples, validate on 196 samples
Epoch 1/100
455/455 [==============================] - 0s 637us/step - loss: 9.6564 - accuracy: 0.4440 - val_loss: 5.5536 - val_accuracy: 0.5408
Epoch 2/100
455/455 [==============================] - 0s 141us/step - loss: 7.1253 - accuracy: 0.5363 - val_loss: 3.6687 - val_accuracy: 0.6786
Epoch 3/100
455/455 [==============================] - 0s 140us/step - loss: 5.9196 - accuracy: 0.5626 - val_loss: 2.7422 - val_accuracy: 0.6786
Epoch 4/100
455/455 [==============================] - 0s 215us/step - loss: 5.0933 - accuracy: 0.5802 - val_loss: 2.1373 - val_accuracy: 0.6633
Epoch 5/100
455/455 [==============================] - 0s 215us/step - loss: 4.4889 - accuracy: 0.6330 - val_loss: 1.6800 - val_accuracy: 0.7704
Epoch 6/100
455/455 [==============================] - 0s 179us/step - loss: 3.5551 - accuracy: 0.6549 - val_loss: 1.4166 - val_accuracy: 0.7653
Epoch 7/100
455/455 [==============================] - 0s 134us/step - loss: 3.4445 

Epoch 57/100
455/455 [==============================] - 0s 104us/step - loss: 1.5534 - accuracy: 0.7890 - val_loss: 0.7059 - val_accuracy: 0.9439
Epoch 58/100
455/455 [==============================] - 0s 102us/step - loss: 1.4940 - accuracy: 0.7912 - val_loss: 0.6722 - val_accuracy: 0.9541
Epoch 59/100
455/455 [==============================] - 0s 96us/step - loss: 1.2856 - accuracy: 0.8022 - val_loss: 0.6419 - val_accuracy: 0.9490
Epoch 60/100
455/455 [==============================] - 0s 99us/step - loss: 1.5866 - accuracy: 0.7736 - val_loss: 0.6333 - val_accuracy: 0.9490
Epoch 61/100
455/455 [==============================] - 0s 91us/step - loss: 1.3710 - accuracy: 0.7736 - val_loss: 0.7000 - val_accuracy: 0.9439
Epoch 62/100
455/455 [==============================] - 0s 105us/step - loss: 1.3863 - accuracy: 0.7516 - val_loss: 0.6631 - val_accuracy: 0.9490
Epoch 63/100
455/455 [==============================] - 0s 93us/step - loss: 1.4764 - accuracy: 0.7758 - val_loss: 0.6765 - val

In [92]:
acc_test_over6 = model1_over6.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over6*100))

196/196 [==============================] - 0s 103us/step
over-sampling test accuracy: 97.96%


In [86]:
pred6 = model1_over6.predict_classes(X_test_over)
pred6

array([0, 1, 0, 2, 0, 2, 0, 1, 1, 1, 2, 1, 2, 0, 0, 0, 2, 2, 2, 1, 2, 1,
       0, 1, 0, 0, 0, 1, 2, 1, 2, 1, 2, 2, 1, 2, 2, 2, 0, 2, 1, 2, 1, 1,
       2, 2, 2, 1, 1, 1, 1, 2, 0, 0, 1, 0, 1, 0, 2, 0, 2, 1, 1, 0, 0, 1,
       1, 2, 2, 1, 0, 2, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 2, 0, 0, 0, 1, 0,
       1, 2, 1, 1, 2, 1, 1, 0, 0, 2, 0, 2, 1, 1, 2, 2, 1, 1, 1, 1, 0, 2,
       2, 1, 0, 1, 2, 0, 0, 0, 2, 1, 2, 1, 1, 2, 2, 0, 0, 2, 2, 0, 1, 0,
       1, 1, 0, 2, 0, 2, 1, 0, 2, 0, 2, 1, 2, 1, 2, 1, 0, 2, 0, 2, 2, 2,
       2, 2, 0, 0, 2, 1, 0, 0, 0, 1, 2, 0, 0, 1, 1, 0, 0, 2, 1, 0, 1, 1,
       1, 2, 2, 2, 1, 2, 0, 2, 1, 1, 2, 2, 1, 2, 1, 0, 1, 1, 0, 2])

In [87]:
dat6['pred'] = pred6
dat6

,0,test,pred
0,120335,0,0
1,NRS168,1,1
2,NRS202,0,0
3,NRS109,2,2
4,BCH-SA-01,0,0
...,...,...,...
191,NRS272,0,0
192,NRS112,1,1
193,NRS064,1,1
194,BCH-SA-04,0,0


In [88]:
proba6 = model1_over6.predict_proba(X_test_over)
dat_proba6 = pd.DataFrame(proba6)

In [89]:
dat_proba6

,0,1,2
0,9.999977e-01,1.597207e-06,7.331805e-07
1,4.360944e-08,1.000000e+00,1.874649e-08
2,9.280388e-01,7.177861e-02,1.825427e-04
3,8.097152e-08,7.371120e-08,9.999999e-01
4,9.999968e-01,2.608138e-06,5.698141e-07
...,...,...,...
191,9.999607e-01,3.367024e-05,5.776848e-06
192,8.275442e-08,9.999999e-01,3.739556e-09
193,2.168245e-08,1.000000e+00,9.603962e-09
194,1.000000e+00,1.026408e-15,1.630406e-14


In [90]:
dat_proba6.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba6.csv", index = False,
         header=None)

In [91]:
dat6.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/6pyopST.csv", index = False,
         header=None)

In [96]:
hist1_over6 = model1_over6.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 455 samples, validate on 196 samples
Epoch 1/100
455/455 [==============================] - 0s 178us/step - loss: 1.1909 - accuracy: 0.8000 - val_loss: 0.5928 - val_accuracy: 0.9796
Epoch 2/100
455/455 [==============================] - 0s 181us/step - loss: 1.1061 - accuracy: 0.8110 - val_loss: 0.5806 - val_accuracy: 0.9796
Epoch 3/100
455/455 [==============================] - 0s 173us/step - loss: 1.0102 - accuracy: 0.8154 - val_loss: 0.6028 - val_accuracy: 0.9745
Epoch 4/100
455/455 [==============================] - 0s 178us/step - loss: 1.1831 - accuracy: 0.7846 - val_loss: 0.5946 - val_accuracy: 0.9796
Epoch 5/100
455/455 [==============================] - 0s 153us/step - loss: 0.9551 - accuracy: 0.8198 - val_loss: 0.5588 - val_accuracy: 0.9796
Epoch 6/100
455/455 [==============================] - 0s 163us/step - loss: 1.2769 - accuracy: 0.7714 - val_loss: 0.5739 - val_accuracy: 0.9796
Epoch 7/100
455/455 [==============================] - 0s 157us/step - loss: 1.0062 

Epoch 57/100
455/455 [==============================] - 0s 174us/step - loss: 0.7608 - accuracy: 0.7846 - val_loss: 0.5080 - val_accuracy: 0.9796
Epoch 58/100
455/455 [==============================] - 0s 220us/step - loss: 0.6826 - accuracy: 0.8286 - val_loss: 0.4018 - val_accuracy: 1.0000
Epoch 59/100
455/455 [==============================] - 0s 158us/step - loss: 0.6888 - accuracy: 0.8132 - val_loss: 0.4042 - val_accuracy: 0.9949
Epoch 60/100
455/455 [==============================] - 0s 146us/step - loss: 0.6781 - accuracy: 0.8484 - val_loss: 0.4912 - val_accuracy: 0.9694
Epoch 61/100
455/455 [==============================] - 0s 149us/step - loss: 0.7741 - accuracy: 0.7802 - val_loss: 0.4826 - val_accuracy: 0.9745
Epoch 62/100
455/455 [==============================] - 0s 150us/step - loss: 0.7588 - accuracy: 0.8000 - val_loss: 0.5111 - val_accuracy: 0.9745
Epoch 63/100
455/455 [==============================] - 0s 142us/step - loss: 0.6445 - accuracy: 0.8132 - val_loss: 0.5071 -

In [97]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over6.history['accuracy'])*100))

over-sampling train accuracy: 79.71%


In [69]:
df_proba6 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_regularizor_dropout_2.xlsx",
                        sheet_name=1,
                        index_col=None)

In [70]:
df_proba6

,phage,strain,phenotype,prediction,0,1,2
0,p002ykpresabsSTCC_qual,NRS209,2,2,1.790400e-08,4.141849e-08,1.000000e+00
1,p002ykpresabsSTCC_qual,NRS386,1,1,5.739934e-04,9.994259e-01,6.773014e-08
2,p002ykpresabsSTCC_qual,NRS148,2,2,5.286934e-09,1.269109e-08,1.000000e+00
3,p002ykpresabsSTCC_qual,NRS178,0,1,6.494936e-12,1.000000e+00,2.537080e-25
4,p002ykpresabsSTCC_qual,NRS237,0,1,5.701098e-02,9.399204e-01,3.068583e-03
...,...,...,...,...,...,...,...
1977,pyopresabsSTCC_qual,NRS272,0,0,9.999607e-01,3.367024e-05,5.776848e-06
1978,pyopresabsSTCC_qual,NRS112,1,1,8.275442e-08,9.999999e-01,3.739556e-09
1979,pyopresabsSTCC_qual,NRS064,1,1,2.168245e-08,1.000000e+00,9.603962e-09
1980,pyopresabsSTCC_qual,BCH-SA-04,0,0,1.000000e+00,1.026408e-15,1.630406e-14


In [71]:
y_prob6 = df_proba6[df_proba6['phage']=='pyopresabsSTCC_qual'].iloc[:,-3:]
y_prob6 = y_prob6.to_numpy()
y_prob6

array([[9.99997740e-01, 1.59720680e-06, 7.33180500e-07],
       [4.36094360e-08, 1.00000000e+00, 1.87464920e-08],
       [9.28038840e-01, 7.17786100e-02, 1.82542660e-04],
       [8.09715200e-08, 7.37112000e-08, 9.99999900e-01],
       [9.99996800e-01, 2.60813840e-06, 5.69814060e-07],
       [8.09715200e-08, 7.37112000e-08, 9.99999900e-01],
       [9.99999760e-01, 2.50527820e-07, 2.58507920e-08],
       [1.46019660e-02, 9.85232500e-01, 1.65560920e-04],
       [2.71841870e-06, 9.99996200e-01, 1.01948390e-06],
       [4.69290070e-04, 9.99520900e-01, 9.77271900e-06],
       [8.09715200e-08, 7.37112000e-08, 9.99999900e-01],
       [3.69294750e-07, 9.99999640e-01, 4.03836000e-09],
       [2.55030800e-08, 2.42283190e-08, 1.00000000e+00],
       [1.00000000e+00, 2.09545130e-30, 1.29175110e-26],
       [9.99958300e-01, 3.80289130e-05, 3.70608000e-06],
       [1.00000000e+00, 1.20991610e-09, 2.60884300e-10],
       [2.55030800e-08, 2.42283190e-08, 1.00000000e+00],
       [2.55030800e-08, 2.42283

In [72]:
ovo6 = rocauc_ovo(y_test_over, y_prob6, average="macro", multi_class="ovo")
ovo6

0.9972294880691828

In [73]:
ovr6 = rocauc_ovr(y_test_over, y_prob6, average="macro", multi_class="ovr")
ovr6

0.9972294880691828

In [74]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=789,
                                                    stratify=y_over)

In [75]:
dat7 = pd.DataFrame(X_test_over[:,0])
dat7['test'] = y_test_over

In [76]:
dat7

,0,test
0,NRS253,1
1,NRS148,2
2,NRS105,1
3,NRS265,1
4,NRS211,0
...,...,...
191,NRS035,0
192,NRS260,1
193,CA9,0
194,NRS183,1


In [77]:
X_train_over = X_train_over[:,1:]
X_test_over = X_test_over[:,1:]

In [96]:
model1_over7 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],), activity_regularizer=l1(0.001)),
    Dense(3, activation='softmax'),
    Dropout(0.2, ),
])

In [97]:
model1_over7.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [98]:
model1_over7.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 455 samples, validate on 196 samples
Epoch 1/100
455/455 [==============================] - 0s 764us/step - loss: 9.5469 - accuracy: 0.4286 - val_loss: 6.6782 - val_accuracy: 0.6224
Epoch 2/100
455/455 [==============================] - 0s 183us/step - loss: 7.0803 - accuracy: 0.5319 - val_loss: 4.2874 - val_accuracy: 0.6480
Epoch 3/100
455/455 [==============================] - 0s 247us/step - loss: 4.9401 - accuracy: 0.6044 - val_loss: 2.6827 - val_accuracy: 0.6480
Epoch 4/100
455/455 [==============================] - 0s 182us/step - loss: 4.2103 - accuracy: 0.6330 - val_loss: 1.9076 - val_accuracy: 0.6939
Epoch 5/100
455/455 [==============================] - 0s 234us/step - loss: 3.9589 - accuracy: 0.6220 - val_loss: 1.1853 - val_accuracy: 0.7806
Epoch 6/100
455/455 [==============================] - 0s 232us/step - loss: 3.3772 - accuracy: 0.6813 - val_loss: 1.0761 - val_accuracy: 0.7857
Epoch 7/100
455/455 [==============================] - 0s 121us/step - loss: 3.2764 

Epoch 57/100
455/455 [==============================] - 0s 153us/step - loss: 1.1855 - accuracy: 0.7692 - val_loss: 1.0664 - val_accuracy: 0.8878
Epoch 58/100
455/455 [==============================] - 0s 141us/step - loss: 1.2389 - accuracy: 0.7604 - val_loss: 1.1040 - val_accuracy: 0.8827
Epoch 59/100
455/455 [==============================] - 0s 137us/step - loss: 1.1830 - accuracy: 0.7802 - val_loss: 0.9404 - val_accuracy: 0.9031
Epoch 60/100
455/455 [==============================] - 0s 132us/step - loss: 1.3767 - accuracy: 0.7846 - val_loss: 0.9908 - val_accuracy: 0.9031
Epoch 61/100
455/455 [==============================] - 0s 115us/step - loss: 1.1689 - accuracy: 0.8132 - val_loss: 0.7894 - val_accuracy: 0.9082
Epoch 62/100
455/455 [==============================] - 0s 110us/step - loss: 1.2635 - accuracy: 0.8022 - val_loss: 0.9391 - val_accuracy: 0.9031
Epoch 63/100
455/455 [==============================] - 0s 180us/step - loss: 1.1363 - accuracy: 0.7868 - val_loss: 1.0220 -

In [105]:
acc_test_over7 = model1_over7.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over7*100))

196/196 [==============================] - 0s 92us/step
over-sampling test accuracy: 92.35%


In [99]:
pred7 = model1_over7.predict_classes(X_test_over)
pred7

array([1, 2, 1, 1, 0, 2, 0, 1, 2, 1, 0, 0, 0, 1, 1, 0, 2, 1, 0, 1, 1, 0,
       1, 0, 2, 1, 2, 2, 1, 1, 1, 0, 1, 2, 0, 2, 2, 0, 1, 1, 2, 1, 1, 2,
       2, 0, 1, 2, 2, 1, 1, 2, 0, 1, 2, 2, 1, 1, 2, 0, 0, 1, 2, 2, 2, 1,
       1, 1, 2, 0, 0, 0, 0, 0, 2, 1, 0, 1, 2, 0, 2, 1, 1, 1, 1, 0, 2, 2,
       2, 2, 2, 1, 1, 2, 2, 1, 0, 2, 1, 0, 2, 0, 2, 1, 1, 1, 0, 1, 0, 0,
       2, 2, 1, 2, 0, 0, 2, 2, 2, 0, 1, 1, 2, 2, 0, 1, 1, 0, 2, 0, 1, 2,
       1, 2, 1, 0, 1, 0, 2, 0, 0, 2, 1, 0, 2, 1, 1, 1, 1, 0, 2, 2, 0, 0,
       0, 2, 2, 1, 1, 2, 1, 2, 1, 1, 2, 2, 1, 1, 1, 2, 1, 2, 2, 2, 1, 2,
       2, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 2, 0, 1, 0, 1, 2])

In [100]:
dat7['pred'] = pred7
dat7

,0,test,pred
0,NRS253,1,1
1,NRS148,2,2
2,NRS105,1,1
3,NRS265,1,1
4,NRS211,0,0
...,...,...,...
191,NRS035,0,0
192,NRS260,1,1
193,CA9,0,0
194,NRS183,1,1


In [101]:
proba7 = model1_over7.predict_proba(X_test_over)
dat_proba7 = pd.DataFrame(proba7)

In [102]:
dat_proba7

,0,1,2
0,8.812649e-10,1.000000e+00,5.528847e-09
1,2.386493e-08,2.366233e-08,1.000000e+00
2,3.804154e-06,9.999962e-01,1.350957e-08
3,4.566996e-08,9.999999e-01,8.425955e-08
4,9.995899e-01,4.056348e-04,4.490389e-06
...,...,...,...
191,9.354528e-01,6.414209e-02,4.051120e-04
192,4.808470e-08,1.000000e+00,7.364639e-09
193,1.000000e+00,2.361323e-08,2.871247e-08
194,2.755864e-07,9.999998e-01,5.310879e-08


In [103]:
dat_proba7.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba7.csv", index = False,
         header=None)

In [104]:
dat7.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/7pyopST.csv", index = False,
         header=None)

In [109]:
hist1_over7 = model1_over7.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 455 samples, validate on 196 samples
Epoch 1/100
455/455 [==============================] - 0s 161us/step - loss: 0.9064 - accuracy: 0.7934 - val_loss: 0.9681 - val_accuracy: 0.9133
Epoch 2/100
455/455 [==============================] - 0s 141us/step - loss: 0.9563 - accuracy: 0.8154 - val_loss: 0.9837 - val_accuracy: 0.9133
Epoch 3/100
455/455 [==============================] - 0s 132us/step - loss: 1.0675 - accuracy: 0.7868 - val_loss: 1.0689 - val_accuracy: 0.9184
Epoch 4/100
455/455 [==============================] - 0s 130us/step - loss: 0.9615 - accuracy: 0.8044 - val_loss: 0.9408 - val_accuracy: 0.9133
Epoch 5/100
455/455 [==============================] - 0s 138us/step - loss: 0.9252 - accuracy: 0.8352 - val_loss: 0.7681 - val_accuracy: 0.9439
Epoch 6/100
455/455 [==============================] - 0s 145us/step - loss: 1.0736 - accuracy: 0.7912 - val_loss: 0.9733 - val_accuracy: 0.9388
Epoch 7/100
455/455 [==============================] - 0s 143us/step - loss: 1.0410 

Epoch 57/100
455/455 [==============================] - 0s 156us/step - loss: 0.7899 - accuracy: 0.7758 - val_loss: 0.7271 - val_accuracy: 0.9541
Epoch 58/100
455/455 [==============================] - 0s 155us/step - loss: 0.7595 - accuracy: 0.7736 - val_loss: 0.7561 - val_accuracy: 0.9490
Epoch 59/100
455/455 [==============================] - 0s 221us/step - loss: 0.7253 - accuracy: 0.8044 - val_loss: 0.6881 - val_accuracy: 0.9490
Epoch 60/100
455/455 [==============================] - 0s 328us/step - loss: 0.6304 - accuracy: 0.8242 - val_loss: 0.8055 - val_accuracy: 0.9439
Epoch 61/100
455/455 [==============================] - 0s 302us/step - loss: 0.7349 - accuracy: 0.7934 - val_loss: 0.6969 - val_accuracy: 0.9490
Epoch 62/100
455/455 [==============================] - 0s 159us/step - loss: 0.6984 - accuracy: 0.8286 - val_loss: 0.6729 - val_accuracy: 0.9388
Epoch 63/100
455/455 [==============================] - 0s 236us/step - loss: 0.6705 - accuracy: 0.8220 - val_loss: 0.7065 -

In [110]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over7.history['accuracy'])*100))

over-sampling train accuracy: 80.38%


In [78]:
df_proba7 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_regularizor_dropout_2.xlsx",
                        sheet_name=2,
                        index_col=None)

In [79]:
df_proba7

,phage,strain,phenotype,prediction,0,1,2
0,p002ykpresabsSTCC_qual,NRS209,2,2,8.300497e-12,1.036520e-09,1.000000e+00
1,p002ykpresabsSTCC_qual,BCH-SA-09,1,1,1.137139e-06,9.999988e-01,2.067601e-09
2,p002ykpresabsSTCC_qual,NRS224,0,0,1.000000e+00,2.093110e-31,0.000000e+00
3,p002ykpresabsSTCC_qual,NRS209,2,2,8.300497e-12,1.036520e-09,1.000000e+00
4,p002ykpresabsSTCC_qual,NRS235,1,1,2.243513e-02,9.774035e-01,1.615106e-04
...,...,...,...,...,...,...,...
1977,pyopresabsSTCC_qual,NRS035,0,0,9.354528e-01,6.414209e-02,4.051121e-04
1978,pyopresabsSTCC_qual,NRS260,1,1,4.808470e-08,1.000000e+00,7.364639e-09
1979,pyopresabsSTCC_qual,CA9,0,0,1.000000e+00,2.361323e-08,2.871247e-08
1980,pyopresabsSTCC_qual,NRS183,1,1,2.755864e-07,9.999998e-01,5.310879e-08


In [80]:
y_prob7 = df_proba7[df_proba7['phage']=='pyopresabsSTCC_qual'].iloc[:,-3:]
y_prob7 = y_prob7.to_numpy()
y_prob7

array([[8.81264950e-10, 1.00000000e+00, 5.52884670e-09],
       [2.38649350e-08, 2.36623300e-08, 1.00000000e+00],
       [3.80415420e-06, 9.99996200e-01, 1.35095730e-08],
       [4.56699600e-08, 9.99999900e-01, 8.42595500e-08],
       [9.99589860e-01, 4.05634780e-04, 4.49038900e-06],
       [6.32779160e-08, 6.94916300e-08, 9.99999900e-01],
       [1.00000000e+00, 2.82767450e-09, 5.89077230e-09],
       [3.59974340e-08, 1.00000000e+00, 2.42576700e-09],
       [1.04225000e-07, 8.56667840e-08, 9.99999760e-01],
       [1.25662545e-02, 9.86555930e-01, 8.77843400e-04],
       [9.59701000e-01, 4.02983200e-02, 6.71130860e-07],
       [1.00000000e+00, 1.92279180e-11, 3.69313200e-11],
       [9.45684130e-01, 5.42763100e-02, 3.95623060e-05],
       [2.75586420e-07, 9.99999760e-01, 5.31087900e-08],
       [2.00877430e-08, 1.00000000e+00, 1.43348300e-08],
       [6.53481960e-01, 3.44687220e-01, 1.83084850e-03],
       [2.38649350e-08, 2.36623300e-08, 1.00000000e+00],
       [1.61404840e-05, 9.99978

In [81]:
ovo7 = rocauc_ovo(y_test_over, y_prob7, average="macro", multi_class="ovo")
ovo7

0.9916422000391466

In [82]:
ovr7 = rocauc_ovr(y_test_over, y_prob7, average="macro", multi_class="ovr")
ovr7

0.9916422000391466

In [83]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=890,
                                                    stratify=y_over)

In [84]:
dat8 = pd.DataFrame(X_test_over[:,0])
dat8['test'] = y_test_over

In [85]:
dat8

,0,test
0,NRS205,2
1,NRS109,2
2,CFBREBSa131,0
3,NRS148,2
4,NRS177,1
...,...,...
191,NRS205,2
192,CFBREBSa122,0
193,NRS001,1
194,NRS148,2


In [86]:
X_train_over = X_train_over[:,1:]
X_test_over = X_test_over[:,1:]

In [109]:
model1_over8 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],), activity_regularizer=l1(0.001)),
    Dense(3, activation='softmax'),
    Dropout(0.2, ),
])

In [110]:
model1_over8.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [111]:
model1_over8.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 455 samples, validate on 196 samples
Epoch 1/100
455/455 [==============================] - 0s 575us/step - loss: 8.2351 - accuracy: 0.4857 - val_loss: 5.2661 - val_accuracy: 0.6276
Epoch 2/100
455/455 [==============================] - 0s 202us/step - loss: 6.5525 - accuracy: 0.5978 - val_loss: 4.2852 - val_accuracy: 0.7041
Epoch 3/100
455/455 [==============================] - 0s 173us/step - loss: 5.9755 - accuracy: 0.6484 - val_loss: 3.2508 - val_accuracy: 0.7347
Epoch 4/100
455/455 [==============================] - 0s 143us/step - loss: 4.9492 - accuracy: 0.6505 - val_loss: 2.6467 - val_accuracy: 0.7857
Epoch 5/100
455/455 [==============================] - 0s 124us/step - loss: 4.4478 - accuracy: 0.7033 - val_loss: 2.1783 - val_accuracy: 0.8061
Epoch 6/100
455/455 [==============================] - 0s 144us/step - loss: 3.8689 - accuracy: 0.7363 - val_loss: 1.8009 - val_accuracy: 0.8061
Epoch 7/100
455/455 [==============================] - 0s 122us/step - loss: 4.0106 

Epoch 57/100
455/455 [==============================] - 0s 145us/step - loss: 1.1703 - accuracy: 0.8220 - val_loss: 0.8837 - val_accuracy: 0.9439
Epoch 58/100
455/455 [==============================] - 0s 149us/step - loss: 1.3728 - accuracy: 0.7582 - val_loss: 0.9150 - val_accuracy: 0.9337
Epoch 59/100
455/455 [==============================] - 0s 145us/step - loss: 1.1771 - accuracy: 0.7736 - val_loss: 0.8678 - val_accuracy: 0.9439
Epoch 60/100
455/455 [==============================] - 0s 133us/step - loss: 1.2649 - accuracy: 0.8044 - val_loss: 0.8690 - val_accuracy: 0.9439
Epoch 61/100
455/455 [==============================] - 0s 128us/step - loss: 1.1060 - accuracy: 0.7912 - val_loss: 0.8825 - val_accuracy: 0.9541
Epoch 62/100
455/455 [==============================] - 0s 163us/step - loss: 1.2150 - accuracy: 0.8088 - val_loss: 0.8602 - val_accuracy: 0.9541
Epoch 63/100
455/455 [==============================] - 0s 175us/step - loss: 1.1494 - accuracy: 0.8066 - val_loss: 0.8675 -

In [118]:
acc_test_over8 = model1_over8.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over8*100))

196/196 [==============================] - 0s 99us/step
over-sampling test accuracy: 96.94%


In [112]:
pred8 = model1_over8.predict_classes(X_test_over)
pred8

array([2, 2, 2, 2, 1, 1, 1, 1, 0, 2, 2, 1, 1, 0, 2, 1, 1, 2, 1, 0, 1, 2,
       0, 1, 1, 0, 2, 0, 0, 2, 0, 0, 2, 2, 2, 0, 1, 1, 1, 2, 2, 2, 2, 1,
       0, 1, 2, 1, 2, 1, 1, 1, 0, 0, 0, 1, 1, 0, 2, 2, 1, 1, 1, 2, 0, 1,
       2, 0, 0, 1, 0, 1, 1, 2, 0, 0, 2, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 2,
       0, 1, 1, 2, 0, 1, 2, 0, 0, 1, 2, 2, 2, 2, 1, 0, 2, 2, 2, 1, 0, 1,
       2, 1, 1, 2, 1, 2, 2, 1, 2, 0, 0, 1, 1, 0, 2, 2, 1, 2, 1, 0, 1, 1,
       0, 1, 1, 2, 0, 0, 2, 1, 0, 0, 0, 2, 2, 2, 2, 2, 1, 1, 1, 1, 0, 0,
       1, 2, 2, 2, 0, 1, 2, 0, 1, 2, 2, 2, 0, 2, 1, 0, 0, 1, 2, 1, 0, 0,
       1, 2, 0, 1, 2, 0, 2, 0, 2, 1, 1, 0, 0, 2, 0, 2, 1, 1, 2, 1])

In [113]:
dat8['pred'] = pred8
dat8

,0,test,pred
0,NRS205,2,2
1,NRS109,2,2
2,CFBREBSa131,0,2
3,NRS148,2,2
4,NRS177,1,1
...,...,...,...
191,NRS205,2,2
192,CFBREBSa122,0,1
193,NRS001,1,1
194,NRS148,2,2


In [114]:
proba8 = model1_over8.predict_proba(X_test_over)
dat_proba8 = pd.DataFrame(proba8)

In [115]:
dat_proba8

,0,1,2
0,3.691095e-08,3.571927e-08,9.999999e-01
1,2.775979e-08,3.876033e-08,9.999999e-01
2,6.982608e-04,1.134029e-06,9.993006e-01
3,3.234670e-08,3.121212e-08,9.999999e-01
4,1.723448e-05,9.999820e-01,6.711543e-07
...,...,...,...
191,3.691095e-08,3.571927e-08,9.999999e-01
192,9.261665e-02,9.073822e-01,1.162373e-06
193,4.174278e-07,9.999995e-01,3.254023e-09
194,3.234670e-08,3.121212e-08,9.999999e-01


In [116]:
dat_proba8.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba8.csv", index = False,
         header=None)

In [117]:
dat8.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/8pyopST.csv", index = False,
         header=None)

In [122]:
hist1_over8 = model1_over8.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 455 samples, validate on 196 samples
Epoch 1/100
455/455 [==============================] - 0s 196us/step - loss: 0.8263 - accuracy: 0.8154 - val_loss: 0.6263 - val_accuracy: 0.9694
Epoch 2/100
455/455 [==============================] - 0s 172us/step - loss: 0.7974 - accuracy: 0.7912 - val_loss: 0.6440 - val_accuracy: 0.9694
Epoch 3/100
455/455 [==============================] - 0s 152us/step - loss: 0.7721 - accuracy: 0.7868 - val_loss: 0.6841 - val_accuracy: 0.9694
Epoch 4/100
455/455 [==============================] - 0s 154us/step - loss: 0.7511 - accuracy: 0.7802 - val_loss: 0.6534 - val_accuracy: 0.9694
Epoch 5/100
455/455 [==============================] - 0s 143us/step - loss: 0.8151 - accuracy: 0.7824 - val_loss: 0.6307 - val_accuracy: 0.9541
Epoch 6/100
455/455 [==============================] - 0s 284us/step - loss: 0.7349 - accuracy: 0.8176 - val_loss: 0.6579 - val_accuracy: 0.9694
Epoch 7/100
455/455 [==============================] - 0s 189us/step - loss: 0.7972 

Epoch 57/100
455/455 [==============================] - 0s 334us/step - loss: 0.6835 - accuracy: 0.7846 - val_loss: 0.5597 - val_accuracy: 0.9745
Epoch 58/100
455/455 [==============================] - 0s 243us/step - loss: 0.6172 - accuracy: 0.8110 - val_loss: 0.5496 - val_accuracy: 0.9745
Epoch 59/100
455/455 [==============================] - 0s 255us/step - loss: 0.6218 - accuracy: 0.8066 - val_loss: 0.5646 - val_accuracy: 0.9745
Epoch 60/100
455/455 [==============================] - 0s 255us/step - loss: 0.6812 - accuracy: 0.8110 - val_loss: 0.5773 - val_accuracy: 0.9694
Epoch 61/100
455/455 [==============================] - 0s 441us/step - loss: 0.6136 - accuracy: 0.8154 - val_loss: 0.5392 - val_accuracy: 0.9745
Epoch 62/100
455/455 [==============================] - 0s 260us/step - loss: 0.6656 - accuracy: 0.7429 - val_loss: 0.5211 - val_accuracy: 0.9745
Epoch 63/100
455/455 [==============================] - 0s 145us/step - loss: 0.6256 - accuracy: 0.8044 - val_loss: 0.5304 -

In [123]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over8.history['accuracy'])*100))

over-sampling train accuracy: 80.10%


In [87]:
df_proba8 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_regularizor_dropout_2.xlsx",
                        sheet_name=3,
                        index_col=None)

In [88]:
df_proba8

,phage,strain,phenotype,prediction,0,1,2
0,p002ykpresabsSTCC_qual,CFBREBSa116,0,0,9.676203e-01,3.237956e-02,1.480166e-07
1,p002ykpresabsSTCC_qual,NRS214,0,0,1.000000e+00,6.534852e-11,2.250731e-18
2,p002ykpresabsSTCC_qual,NRS148,2,2,3.948571e-11,2.839096e-07,9.999998e-01
3,p002ykpresabsSTCC_qual,NRS148,2,2,3.948571e-11,2.839096e-07,9.999998e-01
4,p002ykpresabsSTCC_qual,NRS148,2,2,3.948571e-11,2.839096e-07,9.999998e-01
...,...,...,...,...,...,...,...
1977,pyopresabsSTCC_qual,NRS205,2,2,3.691095e-08,3.571927e-08,9.999999e-01
1978,pyopresabsSTCC_qual,CFBREBSa122,0,1,9.261665e-02,9.073822e-01,1.162373e-06
1979,pyopresabsSTCC_qual,NRS001,1,1,4.174278e-07,9.999995e-01,3.254024e-09
1980,pyopresabsSTCC_qual,NRS148,2,2,3.234670e-08,3.121212e-08,9.999999e-01


In [89]:
y_prob8 = df_proba8[df_proba8['phage']=='pyopresabsSTCC_qual'].iloc[:,-3:]
y_prob8 = y_prob8.to_numpy()
y_prob8

array([[3.69109470e-08, 3.57192700e-08, 9.99999900e-01],
       [2.77597860e-08, 3.87603340e-08, 9.99999900e-01],
       [6.98260850e-04, 1.13402870e-06, 9.99300600e-01],
       [3.23467000e-08, 3.12121200e-08, 9.99999900e-01],
       [1.72344840e-05, 9.99982000e-01, 6.71154340e-07],
       [7.32489840e-08, 9.99999900e-01, 1.02121380e-08],
       [2.71599800e-05, 9.99972700e-01, 1.46475060e-07],
       [1.07752820e-04, 9.99878900e-01, 1.34077460e-05],
       [1.00000000e+00, 2.92587400e-11, 3.97843680e-13],
       [3.69109470e-08, 3.57192700e-08, 9.99999900e-01],
       [2.77597860e-08, 3.87603340e-08, 9.99999900e-01],
       [4.78420900e-08, 1.00000000e+00, 2.14936140e-08],
       [4.17427830e-07, 9.99999500e-01, 3.25402350e-09],
       [1.00000000e+00, 4.12077800e-12, 7.81269150e-12],
       [6.59789600e-08, 7.48123100e-08, 9.99999900e-01],
       [5.20418660e-08, 1.00000000e+00, 9.96792400e-10],
       [3.60579730e-08, 1.00000000e+00, 9.31788500e-11],
       [3.23467000e-08, 3.12121

In [90]:
ovo8 = rocauc_ovo(y_test_over, y_prob8, average="macro", multi_class="ovo")
ovo8

0.9931493442943825

In [91]:
ovr8 = rocauc_ovr(y_test_over, y_prob8, average="macro", multi_class="ovr")
ovr8

0.9931493442943825

In [92]:
ovos2 = [ovo5, ovo6, ovo7, ovo8]
np.mean(ovos2)

0.9930755737053448

In [93]:
np.std(ovos2)

0.0026039960769801106

In [94]:
ovrs2 = [ovr5, ovr6, ovr7, ovr8]
np.mean(ovrs2)

0.9930755737053448

In [95]:
np.std(ovrs2)

0.0026039960769801106

In [124]:
accs_reg = [acc_test_over5, acc_test_over6, acc_test_over7, acc_test_over8]

In [125]:
mean_reg = np.mean(accs_reg)
print('over-sampling test accuracy regularization mean: %.2f%%' % (mean_reg*100))

over-sampling test accuracy regularization mean: 96.05%


In [126]:
std_reg = np.std(accs_reg)
print('over-sampling test accuracy regularization standard deviation:', std_reg)

over-sampling test accuracy regularization standard deviation: 0.021758589075746893


In [127]:
accs_train_reg = [np.mean(hist1_over5.history['accuracy']), np.mean(hist1_over6.history['accuracy']), np.mean(hist1_over7.history['accuracy']),
             np.mean(hist1_over8.history['accuracy'])]

In [128]:
mean_train_reg = np.mean(accs_train_reg)
print('over-sampling train accuracy regularization mean: %.2f%%' % (mean_train_reg*100))

over-sampling train accuracy regularization mean: 80.09%


In [129]:
std_train_reg = np.std(accs_train_reg)
print('over-sampling train accuracy regularization standard deviation:', std_train_reg)

over-sampling train accuracy regularization standard deviation: 0.0024480254
